In [2]:
from pprint import pprint as pp
import numpy as np
import cv2

In [91]:
class Line():
  def __init__(self, index, top, down) -> None:
    self.index = index
    self.down = down
    self.top = top
    
  def __getitem__(self, key):
    if key == 0: return self.index
    if key == 1: return self.top
    if key == 2: return self.down
    
  def __repr__(self):
    return ("{ index: " + str(self.index) + ", " +\
            "top: " + str(self.top) + ", "+ \
            "down: " + str(self.down) + "}")

In [92]:
img = cv2.imread(r"../images/input.jpg")

# Convert BGR to HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# define range of black color in HSV
lower_val = np.array([0,0,0])
upper_val = np.array([50,100,100])

# Threshold the HSV image to get only black colors
mask = cv2.inRange(hsv, lower_val, upper_val)

# invert mask to get black symbols on white background
mask_inv = cv2.bitwise_not(mask)

blank = np.ones(img.shape, np.uint8)*255
blank2 = np.ones(img.shape, np.uint8)*255
blank3 = np.ones(img.shape, np.uint8)*255

contours, hier = cv2.findContours(mask_inv,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)

for cnt in contours:
    if cv2.contourArea(cnt) > 800:
        approx = cv2.approxPolyDP(cnt,
                                    0.00005 * cv2.arcLength(cnt, True), True)
        if(len(approx) >= 5):
            cv2.drawContours(blank, [approx], 0, (0, 0, 0), 1)

graph = []

for j in range(mask_inv.shape[1]):
    higher = 0
    lower = 0

    cntBlank = 0
    lines = []

    for i in range(250):
        if mask_inv[i,j] == 255: 
            cntBlank += 1
            if lower == 0:
                continue
        
        if lower == 0: lower = i
        if mask_inv[i,j] != 255: 
            higher = i
            cntBlank = 0
        if cntBlank > 10:
            lines.append( Line(j, higher, lower) )
            lower = 0
            higher = 0
    if len(lines) > 0:
        graph.append( lines[-1])
    lines.clear()
    # cntLine = 0
    # if (higher + lower > 0):
    #     graph.append( (j, higher, lower) )

prev = graph[0]

prev[0]
print(graph[50])

print(img[0][0])

for p in graph[1:]:
    clr = 125*(p[0]%2)

    if ((p[1] <= prev[1] and p[1] >= prev[2]) or\
        (p[2] <= prev[1] and p[2] >= prev[2])):
    
            cv2.line(blank2, (p[0],p[2]), (p[0], p[1]), (clr,clr,clr))
            prev = p
            lastAcc = p[0]
    # else:
    #         print("")

    cv2.line(img, (p.index ,p[2]), (p[0], p[1]), (clr,clr,clr))
    cv2.line(blank3, (p.index,p[2]), (p[0], p[1]), (clr,clr,clr))

cv2.imwrite(r'../images/mask_inv.png',mask_inv)
cv2.imwrite(r'../images/output2.png',img)
cv2.imwrite(r'../images/output.png',blank3)



{ index: 183, top: 161, down: 160}
[ 97 112  98]


True